In [3]:
#imports
import os
import pandas as pd
import numpy as np
from srim import Ion, Target, Layer, TRIM
import matplotlib.pyplot as plt
%matplotlib inline

#construct beam
b_element = ['O']
b_energy = [284.34e6]
w = 1.5

for i in range(len(b_element)):   
    ion = Ion(b_element[i], b_energy[i])

    #construct target
    Al = Layer({
        'Al': {
            'stoich': 1.0,
            'E_d': 25,
            'lattice': 3.0,
            'surface': 3.36
        }}, density=2.702, width=1500)
    mylar = Layer({
        'H': {
            'stoich': 8.0,
            'E_d': 10,
            'lattice': 3.0,
            'surface': 2},
        'C': {
            'stoich': 10.0,
            'E_d': 28,
            'lattice': 3.0,
            'surface': 7.41},
        'O': {
            'stoich': 4.0,
            'E_d': 28,
            'lattice': 3.0,
            'surface': 2}
        }, density=1.4, width=w*1e4)
    Al3 = Layer({
        'Al': {
            'stoich': 1.0,
            'E_d': 25,
            'lattice': 3.0,
            'surface': 3.36
        }}, density=2.702, width=1500)
    mylar2 = Layer({
        'H': {
            'stoich': 8.0,
            'E_d': 10,
            'lattice': 3.0,
            'surface': 2},
        'C': {
            'stoich': 10.0,
            'E_d': 28,
            'lattice': 3.0,
            'surface': 7.41},
        'O': {
            'stoich': 4.0,
            'E_d': 28,
            'lattice': 3.0,
            'surface': 2}
        }, density=1.4, width=w*1e4)
    #Al2 = Layer({
     #   'Al': {
       #     'stoich': 1.0,
        #    'E_d': 25,
         #   'lattice': 3.0,
          #  'surface': 3.36
        #}}, density=2.702, width=1500)
    air = Layer({
        'Ar': {
            'stoich': 0.012827,
            'E_d': 5,
            'lattice': 1.0,
            'surface': 2},
        'N': {
            'stoich': 0.755267,
            'E_d': 28,
            'lattice': 3.0,
            'surface': 2},
        'C': {
            'stoich': 0.000124,
            'E_d': 28,
            'lattice': 3.0,
            'surface': 7.41},
        'O': {
            'stoich': 0.231781,
            'E_d': 28,
            'lattice': 3.0,
            'surface': 2}
        }, density=1.2048e-3, width=2.5e7)
    target = Target([Al,mylar,Al3,mylar2,air])

    #create TRIM run
    calculation = 1
    kwargs  = {"angle_ions":0, "ranges":True, "transmit":True}
    trim = TRIM(target, ion, calculation, 1000,**kwargs)
    srim_executable_directory = 'C://Users/dziubins/Desktop/SRIM'
    SRIM_Outputs = 'C://Users/dziubins/Desktop/SRIM/SRIM Outputs/'
    results = trim.run(srim_executable_directory)

    #save ouput files
    folder = 'ELOSS Test'
    file_directory = os.path.abspath(SRIM_Outputs + folder)
    os.makedirs(file_directory, exist_ok=True)
    TRIM.copy_output_files(srim_executable_directory, file_directory, check_srim_output= True)
    
df = []

for j in range(len(b_element)):
    #read in transmit file
    folder = 'ELOSS Test'
    os.chdir(SRIM_Outputs + folder)
    mainfolder_dir = os.getcwd()
    file = mainfolder_dir+'\\'+'TRANSMIT.txt'
    fo = open(file)
    strings = fo.readlines()

    #massage data to put into pandas
    lines = 12
    del strings[0:lines]
    for line in range(len(strings)):
        split_ion = strings[line].split()  
        s = set(split_ion) #make s a set of cut and pslit ions\
        if 'T' in s:
            split_ion.remove('T')
        if '-' in s:
            split_ion.remove('-')
        strings[line] = [float(p) for p in split_ion]

    #create pandas dataframe    
    a = ['Ion Num', 'Atomic Num', 'Energy(eV)', 'Depth X(A)', 'Lateral-Pos Y(A)', 
                 'Lateral-Pos Z(A)', 'Dir Cos(X)', 'Dir Cos(Y)', 'Dir Cos(Z)' ]
    df.append(pd.DataFrame(strings,columns=a))
    fo.close() 

for k in range(len(b_element)):
    print(b_element[k])
    #average energy
    avg_E = df[k]['Energy(eV)'].mean()
    print("Average Energy after Windows and 2.5mm air:",avg_E/1e6,"MeV")
    print("Percent Energy Loss in Windows and 2.5mm air:", round(( (b_energy[k] - avg_E) / b_energy[k]) * 100, 3), "%")
    print("Energy Straggling (MeV)", (2.35*df[k]['Energy(eV)'].std())/1e6)
    print('--------------------------------')

O
Average Energy after Windows and 2.5mm air: 283.0012769 MeV
Percent Energy Loss in Windows and 2.5mm air: 0.471 %
Energy Straggling (MeV) 0.1412208598616856
--------------------------------
